In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install warnings
!pip install dgl
!pip install texttable

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 21.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
Looking in indexes: https://pypi.org/simple

In [ ]:
#!unzip '/content/drive/MyDrive/unseen_cmist/unseen_test.zip'

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
import random
from torchvision import transforms, datasets

import os
import pickle
from scipy.spatial.distance import cdist
from scipy import ndimage
import numpy as np

import dgl
import torch
import time
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
def sigma(dists, kth=8):
    # Get k-nearest neighbors for each node
    knns = np.partition(dists, kth, axis=-1)[:, kth::-1]

    # Compute sigma and reshape
    sigma = knns.sum(axis=1).reshape((knns.shape[0], 1))/kth
    return sigma + 1e-8 # adding epsilon to avoid zero value of sigma

def compute_adjacency_matrix_images(coord, feat, use_feat=False, kth=8):
    coord = coord.reshape(-1, 2)
    # Compute coordinate distance
    c_dist = cdist(coord, coord)

    if use_feat:
        # Compute feature distance
        f_dist = cdist(feat, feat)
        # Compute adjacency
        A = np.exp(- (c_dist/sigma(c_dist))**2 - (f_dist/sigma(f_dist))**2 )
    else:
        A = np.exp(- (c_dist/sigma(c_dist))**2)

    # Convert to symmetric matrix
    A = 0.5 * A * A.T
    A[np.diag_indices_from(A)] = 0
    return A

def compute_edges_list(A, kth=8+1):
    # Get k-similar neighbor indices for each node
    if 1==1:
        num_nodes = A.shape[0]
        new_kth = num_nodes - kth
        knns = np.argpartition(A, new_kth-1, axis=-1)[:, new_kth:-1]
        knns_d = np.partition(A, new_kth-1, axis=-1)[:, new_kth:-1]
    else:
        knns = np.argpartition(A, kth, axis=-1)[:, kth::-1]
        knns_d = np.partition(A, kth, axis=-1)[:, kth::-1]
    return knns, knns_d
class newCIFARSuperPix(torch.utils.data.Dataset):
    def __init__(self,
                 data_dir,
                 use_mean_px=True,
                 use_coord=True,
                 use_feat_for_graph_construct=False,):

        #self.split = split
        #self.is_test = split.lower() in ['test', 'val']
        with open(data_dir, 'rb') as f:
            self.labels, self.sp_data = pickle.load(f)

        self.use_mean_px = use_mean_px
        self.use_feat_for_graph = use_feat_for_graph_construct
        self.use_coord = use_coord
        self.n_samples = len(self.labels)
        self.img_size = 32

    def precompute_graph_images(self):
        #print('precompute all data for the %s set...' % self.split.upper())
        self.Adj_matrices, self.node_features, self.edges_lists = [], [], []
        for index, sample in enumerate(self.sp_data):
            mean_px, coord = sample[:2]
            coord = coord / self.img_size
            A = compute_adjacency_matrix_images(coord, mean_px, use_feat=self.use_feat_for_graph)
            edges_list, _ = compute_edges_list(A)
            N_nodes = A.shape[0]

            x = None
            if self.use_mean_px:
                x = mean_px.reshape(N_nodes, -1)
            if self.use_coord:
                coord = coord.reshape(N_nodes, 2)
                if self.use_mean_px:
                    x = np.concatenate((x, coord), axis=1)
                else:
                    x = coord
            if x is None:
                x = np.ones(N_nodes, 1)  # dummy features

            self.node_features.append(x)
            self.Adj_matrices.append(A)
            self.edges_lists.append(edges_list)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        g = dgl.DGLGraph()
        g.add_nodes(self.node_features[index].shape[0])
        g.ndata['feat'] = torch.Tensor(self.node_features[index])
        for src, dsts in enumerate(self.edges_lists[index]):
            g.add_edges(src, dsts[dsts!=src])

        return g, self.labels[index]

use_feat_for_graph_construct = False
tt = time.time()
data_with_feat_knn = newCIFARSuperPix("/content/drive/MyDrive/CMINST_data/CMNIST09_60000_75sp_train.pkl",use_feat_for_graph_construct=use_feat_for_graph_construct)

data_with_feat_knn.precompute_graph_images()
training_data = np.load('/content/drive/MyDrive/CMINST_data/colorMNIST09_60000_data.npy')
training_label=np.load('/content/drive/MyDrive/CMINST_data/colorMNIST09_60000_label.npy')

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
import numpy as np
import os.path as osp
import pickle
import torch
import torch.utils
import torch.utils.data
import torch.nn.functional as F
from scipy.spatial.distance import cdist
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import InMemoryDataset,Data
#/content/drive/MyDrive/Colab_Notebooks/mnist08_83_75sp_train.pkl
def compute_adjacency_matrix_images(coord, sigma=0.1):
    coord = coord.reshape(-1, 2)
    dist = cdist(coord, coord)
    A = np.exp(- dist / (sigma * np.pi) ** 2)
    A[np.diag_indices_from(A)] = 0
    return A


def list_to_torch(data):
    for i in range(len(data)):
        if data[i] is None:
            continue
        elif isinstance(data[i], np.ndarray):
            if data[i].dtype == np.bool:
                data[i] = data[i].astype(np.float32)
            data[i] = torch.from_numpy(data[i]).float()
        elif isinstance(data[i], list):
            data[i] = list_to_torch(data[i])
    return data
def process(data_file):
  use_mean_px=True
  use_coord=True
  node_gt_att_threshold=0
  transform=None
  pre_transform=None
  pre_filter=None

  #data_file ='/content/drive/MyDrive/Colab_Notebooks/colorMNIST05_2000_75sp_train.pkl'

  with open(osp.join(data_file), 'rb') as f:
      labels,sp_data = pickle.load(f)

  #use_mean_px = self.use_mean_px
  #self.use_coord = self.use_coord
  n_samples = len(labels)
  img_size = 32
  #node_gt_att_threshold = self.node_gt_att_threshold

  edge_indices,xs,edge_attrs,node_gt_atts,edge_gt_atts = [], [], [], [], []
  data_list = []
  for index, sample in enumerate(sp_data):
      mean_px, coord = sample[:2]
      coord = coord / img_size
      A = compute_adjacency_matrix_images(coord)
      N_nodes = A.shape[0]

      A = torch.FloatTensor((A > 0.1) * A)
      edge_index, edge_attr = dense_to_sparse(A)

      x = None
      if use_mean_px:
          x = mean_px.reshape(N_nodes, -1)
      if use_coord:
          coord = coord.reshape(N_nodes, 2)
          if use_mean_px:
              x = np.concatenate((x, coord), axis=1)
          else:
              x = coord
      if x is None:
          x = np.ones(N_nodes, 1)  # dummy features

      # replicate features to make it possible to test on colored images
      x = np.pad(x, ((0, 0), (2, 0)), 'edge')
      if node_gt_att_threshold == 0:
          node_gt_att = (mean_px > 0).astype(np.float32)
      else:
          node_gt_att = mean_px.copy()
          node_gt_att[node_gt_att < node_gt_att_threshold] = 0

      node_gt_att = torch.LongTensor(node_gt_att).view(-1)
      row, col = edge_index
      edge_gt_att = torch.LongTensor(node_gt_att[row] * node_gt_att[col]).view(-1)

      data_list.append(
          Data(
              x=torch.tensor(x),
              y=torch.LongTensor([labels[index]]),
              edge_index=edge_index,
              edge_attr=edge_attr,
              node_gt_att=node_gt_att,
              edge_gt_att=edge_gt_att

          )
      )

  #torch.save(InMemoryDataset.collate(data_list), '/content/drive/MyDrive/Colab_Notebooks/colorMINST05_2000.pt')
  return data_list

In [4]:
train_dir='/content/drive/MyDrive/CMINST_data/CMNIST09_10000_75sp_train.pkl'
val_dir='/content/drive/MyDrive/CMINST_data/CMNIST5000_75sp_val.pkl'
test_dir='/content/drive/MyDrive/CMINST_data/CMNIST10000_75sp_test.pkl'
training_final=process(data_file=train_dir)
valing_final=process(data_file=val_dir)
testing_final=process(data_file=test_dir)


In [5]:
import torch
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch_geometric.nn.inits import glorot, zeros
import pdb
def mask_graph(graph_x,select_node):
  mask_value=np.array([0.0001]*32)
  result=np.array([t.detach().numpy() for t in graph_x])
  for i in range(graph_x.shape[0]):
    if(i in select_node):
      continue
    result[i]=mask_value
  return torch.tensor(result)





def split_graph(graph_x,node_of_graph,type_of_graph=True):
  if(type_of_graph==True):
    select_node_number=int(node_of_graph/3)
    select_node=torch.topk(graph_x.mean(axis=1),select_node_number)[1]
    #print(select_node)
    return mask_graph(graph_x,select_node),select_node
  else:
    select_node_number=int(node_of_graph/3*2)
    select_node=torch.topk(graph_x.mean(axis=1),select_node_number)[1]
    return mask_graph(graph_x,select_node),select_node

def uncertainty_mask_gnerate(node_in_graph,number_of_mask):
  all_mask=[]
  for i in range(number_of_mask):
    random_mask=random.sample(node_in_graph.tolist(),int(len(node_in_graph)/3))
    all_mask.append(random_mask)
  return all_mask
def mean(l):
  return sum(l)/len(l)
class GCNConv(MessagePassing):

    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True,
                 edge_norm=True,
                 gfn=False):
        super(GCNConv, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.edge_norm = edge_norm
        self.gfn = gfn
        self.message_mask = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)

        edge_weight = edge_weight.view(-1)


        assert edge_weight.size(0) == edge_index.size(1)

        edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        # Add edge_weight for loop edges.
        loop_weight = torch.full((num_nodes, ),
                                 1 if not improved else 2,
                                 dtype=edge_weight.dtype,
                                 device=edge_weight.device)
        edge_weight = torch.cat([edge_weight, loop_weight], dim=0)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""

        x = torch.matmul(x, self.weight)
        if self.gfn:
            return x

        if not self.cached or self.cached_result is None:
            if self.edge_norm:
                edge_index, norm = GCNConv.norm(
                    edge_index,
                    x.size(0),
                    edge_weight,
                    self.improved,
                    x.dtype)
            else:
                norm = None
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):

        if self.edge_norm:
            return norm.view(-1, 1) * x_j
        else:
            return x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [6]:
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Sequential, ReLU
from torch_geometric.nn import global_mean_pool, global_add_pool, GINConv, GATConv

import random
import pdb
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

layers=3
with_random=True
fc_num=222
hidden=32
eval_random=False
class causalpreCO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreCO, self).__init__()
    num_conv_layers = layers
    self.global_pool = global_add_pool

    self.with_random = with_random
    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)


    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num

    self.object_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )


    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias,0.0001)

  def forward(self,causal,edge_index,causal_edge_weight,batch,eval_random=True):


    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    #xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    #return xc_logis, xo_logis, xco_logis
    return self.objects_readout_layer(xo,batch)




  def objects_readout_layer(self,x,batch):
      xo = self.global_pool(x, batch)
      x_logis = F.log_softmax(self.object_mlp(xo), dim=-1)
      return x_logis



In [7]:
class causalpreNO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreNO, self).__init__()
    num_conv_layers = layers

    self.global_pool = global_add_pool

    self.with_random = with_random

    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)

    self.context_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )

    self.random_readout_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )
    #else:
      #   assert False

    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0.0001)

  #def forward(self,causal,noncausal,batch,causal_edge_weight,noncausal_edge_weight,edge_index,eval_random=True):
  def forward(self,causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random=True):

    xc = F.relu(self.context_convs(self.bnc(noncausal), edge_index, noncausal_edge_weight))
    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    xc = self.global_pool(xc, batch)
    xo = self.global_pool(xo, batch)

    xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    return xc_logis, xco_logis


  def context_readout_layer(self, x):


      x_logis = F.log_softmax(self.context_mlp(x), dim=-1)
      return x_logis

  def random_readout_layer(self, xc, xo, eval_random):

      num = xc.shape[0]
      l = [i for i in range(num)]
      if self.with_random:
          if eval_random:
              random.shuffle(l)
      random_idx = torch.tensor(l)
      #if self.args.cat_or_add == "cat":
      #    x = torch.cat((xc[random_idx], xo), dim=1)
      #else:
      x = xc[random_idx] + xo

      x_logis = F.log_softmax(self.random_readout_mlp(x), dim=-1)
      return x_logis


In [8]:
class causalpreNO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreNO, self).__init__()
    num_conv_layers = layers

    self.global_pool = global_add_pool

    self.with_random = with_random

    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)

    self.context_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )

    self.random_readout_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )
    #else:
      #   assert False

    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0.0001)

  #def forward(self,causal,noncausal,batch,causal_edge_weight,noncausal_edge_weight,edge_index,eval_random=True):
  def forward(self,causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random=True):

    xc = F.relu(self.context_convs(self.bnc(noncausal), edge_index, noncausal_edge_weight))
    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    xc = self.global_pool(xc, batch)
    xo = self.global_pool(xo, batch)

    xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    return xc_logis, xco_logis


  def context_readout_layer(self, x):


      x_logis = F.log_softmax(self.context_mlp(x), dim=-1)
      return x_logis

  def random_readout_layer(self, xc, xo, eval_random):

      num = xc.shape[0]
      l = [i for i in range(num)]
      if self.with_random:
          if eval_random:
              random.shuffle(l)
      random_idx = torch.tensor(l)
      #if self.args.cat_or_add == "cat":
      #    x = torch.cat((xc[random_idx], xo), dim=1)
      #else:
      x = xc[random_idx] + xo

      x_logis = F.log_softmax(self.random_readout_mlp(x), dim=-1)
      return x_logis


In [9]:
class CausalGAN(torch.nn.Module):
    """GCN with BN and residual connection."""
    def __init__(self, num_features,
                       num_classes,
                       gfn=False,
                       collapse=False,
                       residual=False,
                       res_branch="BNConvReLU",
                       global_pool="sum",
                       dropout=0.2,
                       edge_norm=True):
        super(CausalGAN, self).__init__()
        num_conv_layers = layers
        #hidden = args.hidden
        #self.args = args
        hidden=32
        head=4
        self.global_pool = global_add_pool
        self.dropout = dropout
        self.with_random = with_random
        self.without_node_attention = False
        self.without_edge_attention = False
        GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

        hidden_in = num_features
        self.num_classes = num_classes
        hidden_out = num_classes
        self.fc_num = fc_num
        self.bn_feat = BatchNorm1d(hidden_in)
        self.conv_feat = GCNConv(hidden_in, hidden, gfn=True) # linear transform
        self.bns_conv = torch.nn.ModuleList()
        self.convs = torch.nn.ModuleList()

        for i in range(num_conv_layers):
            self.bns_conv.append(BatchNorm1d(hidden))
            self.convs.append(GATConv(hidden, int(hidden / head), heads=head, dropout=dropout))

        self.edge_att_mlp = nn.Linear(hidden * 2, 2)
        self.node_att_mlp = nn.Linear(hidden, 2)
        self.bnc = BatchNorm1d(hidden)
        self.bno= BatchNorm1d(hidden)
        self.context_convs = GConv(hidden, hidden)
        self.objects_convs = GConv(hidden, hidden)
        self.relu = nn.ReLU(inplace=False)

        for m in self.modules():
            if isinstance(m, (torch.nn.BatchNorm1d)):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0.0001)

    def forward(self, data, eval_random=True):

        x = data.x if data.x is not None else data.feat
        edge_index,edge_attr,batch = data.edge_index,data.edge_attr,data.batch
        row, col = edge_index
        x = self.bn_feat(x)
        x = self.relu(self.conv_feat(x, edge_index,edge_weight=edge_attr))

        for i, conv in enumerate(self.convs):
            x = self.bns_conv[i](x)
            x = self.relu(conv(x, edge_index))

        edge_rep = torch.cat([x[row], x[col]], dim=-1)

        if self.without_edge_attention:
            edge_att = 0.5 * torch.ones(edge_rep.shape[0], 2).to(device)
        else:
            edge_att = F.softmax(self.edge_att_mlp(edge_rep), dim=-1)
        edge_weight_c = edge_att[:, 0]
        edge_weight_o = edge_att[:, 1]

        if self.without_node_attention:
            node_att = 0.5 * torch.ones(x.shape[0], 2).to(device)
        else:
            node_att = F.softmax(self.node_att_mlp(x), dim=-1)
        xc = node_att[:, 0].view(-1, 1) * x
        xo = node_att[:, 1].view(-1, 1) * x

        #print(edge_weight_o)
        #xc = F.relu(self.context_convs(self.bnc(xc), edge_index, edge_weight_c))
        #xo = F.relu(self.objects_convs(self.bno(xo), edge_index, edge_weight_o))
        #node_of_graph=xo.shape[0]
        #causal_part,causal_node=split_graph(graph_x=xo,node_of_graph=node_of_graph)
        #noncausal_part,noncausal_node=split_graph(graph_x=xc,node_of_graph=node_of_graph,type_of_graph=False)

        #xc = self.global_pool(xc, batch)
        #xo = self.global_pool(xo, batch)

        #xc_logis = self.context_readout_layer(xc)
        #xo_logis = self.objects_readout_layer(xo)
        #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

      # return (xo,edge_index,edge_weight_o),(xc,edge_index,edge_weight_c),batch
        return (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch
        #return (causal_attention,causal_part,causal_node),(noncausal_attention,noncausal_part,noncausal_node),batch
        #return (xo,causal_part,causal_node),(xc,noncausal_part,noncausal_node),batch



In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR


In [11]:
train_loader = DataLoader(training_final, batch_size=128, shuffle=True)
val_loader = DataLoader(valing_final, batch_size=128, shuffle=False)
test_loader = DataLoader(testing_final, batch_size=128, shuffle=False)
#t_load=[]
#for i in train_loader:
#  t_load.append(i)
#  if(len(t_load)==10000):
#    break


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [12]:
number_of_class=10
causal_model2 = CausalGAN(7,number_of_class).to(device)
predictno_model2=causalpreNO(7,number_of_class).to(device)
predictco_model2=causalpreCO(7,number_of_class).to(device)
model_optimizer2 = torch.optim.Adam(
            list(causal_model2.parameters()) +
            list(predictno_model2.parameters())+list(predictco_model2.parameters()),
            lr=0.001)
Epo=150
lr_scheduler = CosineAnnealingLR(model_optimizer2, T_max=Epo, eta_min=1e-6, last_epoch=-1, verbose=False)

In [15]:
import time
import json

loss_value=[]
loss_value_valation=[]
c=0.5
o=1
co=0.5
def num_graphs(data):
  if data.batch is not None:
      return data.num_graphs
  else:
      return data.x.size(0)
from tqdm import tqdm

for epoch in range(Epo):
  #model.train()
  causal_model2.train()
  predictno_model2.train()
  predictco_model2.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  correct_o = 0
  nb=0
  print(f"-----training-------{epoch}")
  loop = tqdm(enumerate(train_loader),total=len(train_loader))
  for it, data in loop:
#  for it, data in enumerate(train_loader):
      nb+=1
      model_optimizer2.zero_grad()
      data = data.to(device)

      one_hot_target = data.y.view(-1)
      #causal,noncausal,batch=causal_model(data)
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device)/number_of_class
     # sim=F.cosine_similarity(causal_attention,noncausal_attention).mean()
      #print(o_logs)
      #print(type(o_logs))
      c_loss = F.kl_div(c_logs, uniform_target,reduction='batchmean')
      #print(f'causal loss{c_loss}')
      o_loss = F.nll_loss(o_logs,one_hot_target)
      #print(f'noncausal loss{o_loss}')
      co_loss = F.nll_loss(co_logs,one_hot_target)
      #print(f'com causal loss{co_loss}')
      loss = c * c_loss + o * o_loss + co * co_loss

      pred_o = o_logs.max(1)[1]
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      loss.backward()
      total_loss += loss.item() #* num_graphs(data)
      total_loss_c += c_loss.item() #* num_graphs(data)
      total_loss_o += o_loss.item() #* num_graphs(data)
      total_loss_co += co_loss.item()# * num_graphs(data)
      model_optimizer2.step()
      loop.set_description(f"Epoch [{epoch}/{Epo}]")
      loop.set_postfix(loss = loss.item())

  #num = len(train_loader.dataset)
  lr_scheduler.step()
  total_loss = total_loss / nb
  total_loss_c = total_loss_c / nb
  print(f'number of {epoch} with total loss:{total_loss}')
  loss_value.append(total_loss)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    print(f"------valation---------{epoch}")
    causal_model2.eval()
    predictno_model2.eval()
    predictco_model2.eval()
    for data in val_loader:
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    print(f"causal val accuracy:{acc_co}")
    loss_value_valation.append(acc_co)
    dictionary={"number of epoch":epoch,
                "training loss list":loss_value,
                "valation accuracy list":loss_value_valation}

    # Serializing json
    json_object = json.dumps(dictionary,indent=3)

    # Writing to sample.json
   # with open("/content/drive/MyDrive/running_cal_mnist/number_tl_va_e08.json", "w") as outfile:
   #     outfile.write(json_object)

    torch.save({
            'causal_model.state_dic': causal_model2.state_dict(),
            'predictco_model_state_dict()': predictco_model2.state_dict(),
            'predictno_model_state_dict()': predictno_model2.state_dict(),
            'opt':model_optimizer2.state_dict()
            }, '/content/drive/MyDrive/running_cal_mnist/allmodel_cal_0933.pt')
    if(epoch>50):
      check=abs(acc_o-loss_value_valation[len(loss_value_valation)-20])/20
      if(check<=0.01):
        break




-----training-------0


Epoch [0/150]: 100%|██████████| 79/79 [00:29<00:00,  2.65it/s, loss=0.701]


number of 0 with total loss:0.3561730047192755
------valation---------0
causal val accuracy:0.2542
-----training-------1


Epoch [1/150]: 100%|██████████| 79/79 [00:28<00:00,  2.74it/s, loss=0.0903]


number of 1 with total loss:0.34443934940839116
------valation---------1
causal val accuracy:0.2628
-----training-------2


...

number of 49 with total loss:0.2185090932099125
------valation---------49
causal val accuracy:0.3697
-----training-------50


Epoch [50/150]: 100%|██████████| 79/79 [00:29<00:00,  2.72it/s, loss=0.415]


number of 50 with total loss:0.2322648847593537
------valation---------50
causal val accuracy:0.3719
-----training-------51


Epoch [51/150]: 100%|██████████| 79/79 [00:28<00:00,  2.75it/s, loss=0.379]


number of 51 with total loss:0.21857946818765206
------valation---------51
causal val accuracy:0.3942


In [ ]:
aaa

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/running_cal_mnist/allmodel_cal_09.pt')
causal_model2.load_state_dict(checkpoint['causal_model.state_dic'])
predictco_model2.load_state_dict(checkpoint['predictco_model_state_dict()'])
predictno_model2.load_state_dict(checkpoint['predictno_model_state_dict()'])

In [ ]:
test_loader = DataLoader(testing_final, batch_size=1, shuffle=False)

correct=0
correct_c=0
correct_o=0
print(f"------test---------{0000000}")
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()
for data in test_loader:
  (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
  c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
  pred = co_logs.max(1)[1]
  pred_c = c_logs.max(1)[1]
  pred_o = o_logs.max(1)[1]
  correct += pred.eq(data.y.view(-1)).sum().item()
  correct_c += pred_c.eq(data.y.view(-1)).sum().item()
  correct_o += pred_o.eq(data.y.view(-1)).sum().item()
acc_co = correct / len(test_loader.dataset)
acc_c = correct_c / len(test_loader.dataset)
acc_o = correct_o / len(test_loader.dataset)
print(f"combian causal test accuracy:{acc_co}")
print(f"causal test accuracy:{acc_o}")
#dictionary={"number of epoch":epoch,
#            "training loss list":loss_value,
#            "valation accuracy list":loss_value_valation,
#            "test accuracy value":acc_co}

# Serializing json
#json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
#with open("/content/drive/MyDrive/running_cal_mnist/number_tl_va_e09.json", "w") as outfile:
#    outfile.write(json_object)


In [ ]:
test_loader = DataLoader(testing_final[:10], batch_size=1, shuffle=False)

In [ ]:
from torch_geometric.utils import to_dgl,from_dgl,k_hop_subgraph,is_undirected

In [ ]:
tey=tol[0]
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()
(xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(tey)
lab=tey.y
tey

In [ ]:
xo

In [ ]:
import torch
import math
import math
import numpy as np
from torch_geometric.utils import (remove_self_loops, degree,
                                   batched_negative_sampling)
from torch_geometric.utils.num_nodes import maybe_num_nodes
def split_batch(g):
    split = degree(g.batch[g.edge_index[0]], dtype=torch.long).tolist()
    edge_indices = torch.split(g.edge_index, split, dim=1)
    num_nodes = degree(g.batch, dtype=torch.long)
    cum_nodes = torch.cat([g.batch.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]])
    num_edges = torch.tensor([e.size(1) for e in edge_indices], dtype=torch.long).to(g.x.device)
    cum_edges = torch.cat([g.batch.new_zeros(1), num_edges.cumsum(dim=0)[:-1]])

    return edge_indices, num_nodes, cum_nodes, num_edges, cum_edges

def split_graph(data, edge_score, ratio):
    causal_edge_index = torch.LongTensor([[],[]]).to(data.x.device)
    causal_edge_weight = torch.tensor([]).to(data.x.device)
    causal_edge_attr = torch.tensor([]).to(data.x.device)
    conf_edge_index = torch.LongTensor([[],[]]).to(data.x.device)
    conf_edge_weight = torch.tensor([]).to(data.x.device)
    conf_edge_attr = torch.tensor([]).to(data.x.device)

    edge_indices, _, _, num_edges, cum_edges = split_batch(data)
    for edge_index, N, C in zip(edge_indices, num_edges, cum_edges):
        n_reserve =  int(ratio * N)
        edge_attr = data.edge_attr[C:C+N]
        single_mask = edge_score[C:C+N]
        single_mask_detach = edge_score[C:C+N].detach().cpu().numpy()
        rank = np.argpartition(-single_mask_detach, n_reserve)
        idx_reserve, idx_drop = rank[:n_reserve], rank[n_reserve:]

        causal_edge_index = torch.cat([causal_edge_index, edge_index[:, idx_reserve]], dim=1)
        conf_edge_index = torch.cat([conf_edge_index, edge_index[:, idx_drop]], dim=1)

        causal_edge_weight = torch.cat([causal_edge_weight, single_mask[idx_reserve]])
        conf_edge_weight = torch.cat([conf_edge_weight,  -1 * single_mask[idx_drop]])

        causal_edge_attr = torch.cat([causal_edge_attr, edge_attr[idx_reserve]])
        conf_edge_attr = torch.cat([conf_edge_attr, edge_attr[idx_drop]])
    return (causal_edge_index, causal_edge_attr, causal_edge_weight), \
        (conf_edge_index, conf_edge_attr, conf_edge_weight)




In [ ]:
ratio=0.1

causal_edge_index = torch.LongTensor([[],[]])
causal_edge_weight = torch.tensor([])
causal_edge_attr = torch.tensor([])

edge_indices, _, _, num_edges, cum_edges = split_batch(tey)
split_batch(tey)

In [ ]:
torch.topk(xc[non_sub_nodes].mean(axis=1),int(len(non_sub_nodes)*0.1))

In [ ]:
ratio=0.5

causal_edge_index = torch.LongTensor([[],[]])
causal_edge_weight = torch.tensor([])
conf_edge_index = torch.LongTensor([[],[]])
conf_edge_weight = torch.tensor([])

#causal_edge_attr = torch.tensor([])
topnode=torch.LongTensor([])
notopnode=torch.LongTensor([])

edge_indices, _, _, num_edges, cum_edges = split_batch(tey)
for edge_index, N, C in zip(edge_indices, num_edges, cum_edges):
    n_reserve =  int(ratio * N)
    edge_attr = edge_weight_o[C:C+N]
    single_mask = edge_weight_o[C:C+N]
    single_mask_detach = edge_weight_o[C:C+N].detach().cpu().numpy()
    rank = np.argpartition(-single_mask_detach, n_reserve)
    idx_reserve, idx_drop = rank[:n_reserve], rank[n_reserve:]

    causal_edge_index = torch.cat([causal_edge_index, edge_index[:, idx_reserve]], dim=1)
    conf_edge_index = torch.cat([conf_edge_index, edge_index[:, idx_drop]], dim=1)
    #print(causal_edge_index)
    causal_edge_weight = torch.cat([causal_edge_weight, single_mask[idx_reserve]])
    conf_edge_weight = torch.cat([conf_edge_weight,  -1 * single_mask[idx_drop]])
    causal_sub_nodes = torch.unique(edge_index[:, idx_reserve])
    non_sub_nodes=torch.unique(edge_index[:, idx_drop])

    print(f'non_subnode:{non_sub_nodes}')
   # print(f'non weight:{ -1 * single_mask[idx_drop]}')
    print(f'subnode:{causal_sub_nodes}')
    #print(f'c subnode weight:{single_mask[idx_reserve]}')
    topnon=torch.topk(xc[non_sub_nodes].mean(axis=1),int(len(non_sub_nodes)*0.1))[1]
    top=torch.topk(xo[causal_sub_nodes].mean(axis=1),int(len(causal_sub_nodes)*0.1))[1]
    topnode = torch.cat([topnode, causal_sub_nodes[top]])
    notopnode=torch.cat([notopnode, non_sub_nodes[topnon]])
    #print(topnode)
    print(f'top node:{topnode}')
    print(f'nontop node:{notopnode}')


   #causal_edge_attr = torch.cat([causal_edge_attr, edge_attr[idx_reserve]])
    #conf_edge_attr = torch.cat([conf_edge_attr, edge_attr[idx_drop]])

In [ ]:
def split_graph_bybatch(edge_weight_o,xo,xc,data,ratio=0.5):
  causal_edge_index = torch.LongTensor([[],[]])
  causal_edge_weight = torch.tensor([])
  conf_edge_index = torch.LongTensor([[],[]])
  conf_edge_weight = torch.tensor([])

  #causal_edge_attr = torch.tensor([])
  topnode=torch.LongTensor([])
  notopnode=torch.LongTensor([])

  edge_indices, _, _, num_edges, cum_edges = split_batch(data)
  for edge_index, N, C in zip(edge_indices, num_edges, cum_edges):
      n_reserve =  int(ratio * N)
      edge_attr = edge_weight_o[C:C+N]
      single_mask = edge_weight_o[C:C+N]
      single_mask_detach = edge_weight_o[C:C+N].detach().cpu().numpy()
      rank = np.argpartition(-single_mask_detach, n_reserve)
      idx_reserve, idx_drop = rank[:n_reserve], rank[n_reserve:]

      causal_edge_index = torch.cat([causal_edge_index, edge_index[:, idx_reserve]], dim=1)
      conf_edge_index = torch.cat([conf_edge_index, edge_index[:, idx_drop]], dim=1)
      #print(causal_edge_index)
      causal_edge_weight = torch.cat([causal_edge_weight, single_mask[idx_reserve]])
      conf_edge_weight = torch.cat([conf_edge_weight,  -1 * single_mask[idx_drop]])
      causal_sub_nodes = torch.unique(edge_index[:, idx_reserve])
      non_sub_nodes=torch.unique(edge_index[:, idx_drop])

      #print(f'non_subnode:{non_sub_nodes}')
    # print(f'non weight:{ -1 * single_mask[idx_drop]}')
      #print(f'subnode:{causal_sub_nodes}')
      #print(f'c subnode weight:{single_mask[idx_reserve]}')
      topnon=torch.topk(xc[non_sub_nodes].mean(axis=1),int(len(causal_sub_nodes)*0.08))[1]
      top=torch.topk(xo[causal_sub_nodes].mean(axis=1),int(len(causal_sub_nodes)*0.08))[1]
      topnode = torch.cat([topnode, causal_sub_nodes[top]])
      notopnode=torch.cat([notopnode, non_sub_nodes[topnon]])
      #print(topnode)
      #print(f'top node:{topnode}')
      #print(f'nontop node:{notopnode}')
  return (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)


In [ ]:
(causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)=split_graph_bybatch(edge_weight_o,xo,xc,tey,ratio=0.5)

In [ ]:
is_undirected(causal_edge_index,causal_edge_weight)

In [ ]:
tey.batch[topnode]
tey.y

In [ ]:
from collections import Counter
batchnumber_index

In [ ]:
select_node_batch_number

In [ ]:
number_of_g_inbatch=2
select_node_batch_number=tey.batch[topnode]
tt = []
for i in range(number_of_g_inbatch):
  tt.append((select_node_batch_number == i).unsqueeze(0))
batchnumber_index=torch.cat(tt,dim=0).sum(dim=1)
number_of_sample=int(min(batchnumber_index))
n=0
while(n<number_of_sample):
  n+=1
  for i in select_node_batch_number:




In [ ]:
topnode

In [ ]:
notopnode

In [ ]:
def k_hop_subgraph(node_idx,num_hops,edge_index,ew,relabel_nodes=False,num_nodes = None):

    num_nodes = maybe_num_nodes(edge_index, num_nodes)


    col, row = edge_index

    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    if isinstance(node_idx, (int, list, tuple)):
        node_idx = torch.tensor([node_idx], device=row.device).flatten()
    else:
        node_idx = node_idx.to(row.device)

    subsets = [node_idx]

    for _ in range(num_hops):
        node_mask.fill_(False)
        node_mask[subsets[-1]] = True
        torch.index_select(node_mask, 0, row, out=edge_mask)
        subsets.append(col[edge_mask])

    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    node_mask.fill_(False)
    node_mask[subset] = True


    edge_mask = node_mask[row] & node_mask[col]

    edge_index = edge_index[:, edge_mask]

    edge_weight=torch.masked_select(ew,edge_mask)
    if relabel_nodes:
        node_idx = row.new_full((num_nodes, ), -1)
        node_idx[subset] = torch.arange(subset.size(0), device=row.device)
        edge_index = node_idx[edge_index]

    return subset, edge_index, inv,edge_weight


In [ ]:
subset, edge_index,_,edge_weight=k_hop_subgraph(8,5,causal_edge_index,causal_edge_weight,relabel_nodes=False)
edge_weight

In [ ]:
from torch_geometric.data import Data,Batch

In [ ]:
Data(x=xo[subset],edge_index=edge_index,edge_attr=edge_weight,y=0)

In [ ]:
def combine_subg(databatch,select_node,edge_indexn,eweight,tragety):
  undata=[]
  select_node_batch_number=databatch[select_node]
  for i in range(len(select_node)):
    print(i)
    #subset,edge_index,_,edge_weight=k_hop_subgraph(int(select_node[i]),5,edge_index,eweight,relabel_nodes=False)
    subset, edge_index,_,edge_weight=k_hop_subgraph(int(select_node[i]),5,edge_indexn,eweight,relabel_nodes=False)
    undata.append(Data(x=xo[subset],edge_index=edge_index,edge_attr=edge_weight,y=tragety[int(select_node_batch_number[i])]))
  #d=DataLoader(undata,batch_size=int(len(select_node)/4),shuffle=True)
  fin=Batch.from_data_list(undata)
  return fin


combine_subg(tey.batch,topnode,causal_edge_index,causal_edge_weight,tey.y)

In [ ]:
d=DataLoader(undata,batch_size=len(topnode))
Batch.from_data_list(undata)


In [ ]:
for i in d:
  print(i)

In [ ]:
#xo[subset]
torch.masked_select(causal_edge_weight,edge_mask)

In [ ]:
k_hop_subgraph(16,5,causal_edge_index)[0]

In [ ]:
topnode

In [ ]:
batch[topnode]

In [ ]:
subn, _, _, broken_mask=bid_k_hop_subgraph(16,3, causal_edge_index, relabel_nodes=False,num_nodes=None)

In [ ]:
xo[subn]

In [ ]:
torch.masked_select(causal_edge_weight,broken_mask)

In [ ]:
edge_index

In [ ]:
causal_edge_index

In [ ]:
causal_edge_weight

In [ ]:
causal_edge_weight[edge_mask]

In [ ]:
graph_nodes_number=xo.shape[0]
graph_nodes_number

In [ ]:
xo.sum(axis=1)[48:]

In [ ]:
top10=torch.topk(xo.mean(axis=1),int(graph_nodes_number*0.2))

In [ ]:
top10

In [ ]:
#lab

G = dgl.DGLGraph()
G.add_nodes(graph_nodes_number)
G.ndata['x']=xo
G.add_edges(edge_index[0],edge_index[1])
G.edata['edge_attr']=edge_weight_o
G.ndata['batch']=batch
G

In [ ]:
G.ndata['x']

In [ ]:
offset = 0
nodes_offset = 0
new_data_list = []
#label_list = []
for data, label in zip(data_list, batch_labels):
    num_edges = data.number_of_edges()
    edge_score = data_mask[offset:offset + num_edges]
    mean_score = torch.mean(edge_score)
    sorted_score, index = torch.sort(edge_score.view(-1))

    prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
    #_, index = torch.sort(edge_score.view(-1))
    prune_index = index[:prune_num_edges]
    data.remove_edges(prune_index)
    #data.edata['mask'] = edge_score.cpu()
    isolated_nodes = ((data.in_degrees() == 0) & (data.out_degrees() == 0)).nonzero().squeeze(1)
    data.remove_nodes(isolated_nodes)
    new_data_list.append(data)
    offset += num_edges

In [ ]:
edge_score = edge_weight_o.clone()
mean_score = torch.mean(edge_score)
sorted_score, index = torch.sort(edge_score.view(-1))
prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
prune_index=index[:prune_num_edges]
G.remove_edges(prune_index)


In [ ]:
#tey=to_dgl(tey)

In [ ]:
tey

In [ ]:
tey.edges()

In [ ]:
sg.edges()

In [ ]:
sg, inverse_indices = dgl.khop_out_subgraph(tey, 0, k=1)

In [ ]:
edge_weight_o.view(-1)

In [ ]:
def subg_edge(data,weight):
  re_data=data.clone()
  edge_score = weight.clone()
  re_data=to_dgl(re_data)
  mean_score = torch.mean(edge_score)
  sorted_score, index = torch.sort(edge_score.view(-1))
  prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
  prune_index=index[:prune_num_edges]
  re_data.remove_edges(prune_index)
 # isolated_nodes = ((re_data.in_degrees() == 0) & (re_data.out_degrees() == 0)).nonzero().squeeze(1)
 # re_data.remove_nodes(isolated_nodes)
  re_data=from_dgl(re_data)
  return re_data


In [ ]:
def subg_node(data,top_select_number):
  node_number=data.x.shape[0]
  re_data=data.clone()
  node_weight = data.x.mean(axis=1)
  re_data=to_dgl(re_data)
  #mean_score = torch.mean(edge_score)
  prune_number_node=node_number-top_select_number
  sorted_score, index = torch.sort(node_weight.view(-1))
  #prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
  prune_index=index[:prune_number_node]
  re_data.remove_nodes(prune_index)
  re_data=from_dgl(re_data)
  return re_data


In [ ]:
subg_node(tey,10).edge_index

In [ ]:
sorted_score, index=torch.topk(edge_score.view(-1),10)
index

In [ ]:
num_edges = tey.number_of_edges()
edge_score = edge_weight_o
mean_score = torch.mean(edge_score)
sorted_score, index = torch.sort(edge_score.view(-1))

prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
prune_index=index[:prune_num_edges]


In [ ]:
tey.remove_edges(prune_index)

In [ ]:
isolated_nodes = ((tey.in_degrees() == 0) & (tey.out_degrees() == 0)).nonzero().squeeze(1)


In [ ]:
isolated_nodes

In [ ]:
tey.remove_nodes(isolated_nodes)
tey=from_dgl(tey)
tey

In [ ]:
tey.batch

In [ ]:
(xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(tey)

#c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
pred = co_logs.max(1)[1]
pred_o = o_logs.max(1)[1]
print(pred.eq(lab.view(-1)).sum().item())
print(pred_o)

In [ ]:
pred.eq(lab.view(-1))

In [ ]:
offset = 0
nodes_offset = 0
new_data_list = []
label_list = []
for data, label in zip(data_list, batch_labels):
    num_edges = data.number_of_edges()
    edge_score = edge_weight_o
    mean_score = torch.mean(edge_score)
    sorted_score, index = torch.sort(edge_score.view(-1))

    prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
    #_, index = torch.sort(edge_score.view(-1))
    prune_index = index[:prune_num_edges]
    data.remove_edges(prune_index)
    #data.edata['mask'] = edge_score.cpu()
    isolated_nodes = ((data.in_degrees() == 0) & (data.out_degrees() == 0)).nonzero().squeeze(1)
    data.remove_nodes(isolated_nodes)
    new_data_list.append(data)
    label_list.append(label)
    offset += num_edges

In [ ]:
def pruning_batch_data_from_mask_cp(data_list, batch_labels, data_mask):
    offset = 0
    nodes_offset = 0
    new_data_list = []
    label_list = []
    for data, label in zip(data_list, batch_labels):
        num_edges = data.number_of_edges()
        edge_score = data_mask[offset:offset + num_edges]
        mean_score = torch.mean(edge_score)
        sorted_score, index = torch.sort(edge_score.view(-1))

        prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
        #_, index = torch.sort(edge_score.view(-1))
        prune_index = index[:prune_num_edges]
        data.remove_edges(prune_index)
        #data.edata['mask'] = edge_score.cpu()
        isolated_nodes = ((data.in_degrees() == 0) & (data.out_degrees() == 0)).nonzero().squeeze(1)
        data.remove_nodes(isolated_nodes)
        new_data_list.append(data)
        label_list.append(label)
        offset += num_edges

    return new_data_list, label_list

In [ ]:
(xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(tey)

In [ ]:
edge_weight_o.shape

In [ ]:
torch.topk(edge_weight_o,10)

In [ ]:
tey.edge_index

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
# Opening JSON file
f = open('/content/drive/MyDrive/running_cal_mnist/number_tl_va_e09.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
loss_valuefi=data['training loss list']
for i in range(len(loss_valuefi)):
  loss_valuefi[i]=float(loss_valuefi[i])
plt.plot(np.array(loss_valuefi))
plt.show()